<a href="https://colab.research.google.com/github/yuliiabosher/Fiber-optic-project/blob/europe_stats_analysis/european_fibre_optic_data_clean_and_load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Fibre optic data - European statistical data.
Load data and clean

This notebook contains code to load data provided by the European Commission about broadband connectivity and availability in 27 EU states and the UK. The data will be used to compare fibre optic coverage in the UK with that in key european countries. The data covers several years and also provides a split between rural and urban areas.

After loading the data, there is code to show some high level summative and descriptive stats


Read the data files - location of the files should be updated based on your environment EC Broadband coverage in Europe data file available from https://digital-strategy.ec.europa.eu/en/library/digital-decade-2024-broadband-coverage-europe-2023

Code checks files exist in specified location

NB - currently I have downloaded the excel spreadsheet from the link above and saved the last 2 sheets as csv files

sheet **data** - save as EUROPE_FIBRE_HH

sheet **data%** - save as EUROPE_FIBRE

In [21]:
# Import and alias the necessary libraries
import pandas as pd
import os.path
import errno

#mount google drive
from google.colab import drive
drive.mount('/content/drive')

#############
# EC broadband data file available from https://digital-strategy.ec.europa.eu/en/library/digital-decade-2024-broadband-coverage-europe-2023
# save the last sheet in the spreadsheet as .csv and name EUROPE_FIBRE.csv
# This path should be set to the location of the file
#############
fileEuropeData = '/content/drive/MyDrive/Colab/EUROPE_FIBRE.csv'
fileEuropeHouseholds = '/content/drive/MyDrive/Colab/EUROPE_FIBRE_HH.csv'

#fileEuropeData = 'd:/Users/Sharon/Documents/College/data/EUROPE_FIBRE.csv'
#fileEuropeHouseholds = 'd:/Users/Sharon/Documents/College/data/EUROPE_FIBRE_HH.csv'
#############
# Check we can find the file required, and read it into a pandas dataframe
# show the shape of the dataframe
#############
if os.path.exists(fileEuropeData) :
    print("Reading Europe fibre data.....")
    dfEurope = pd.read_csv(fileEuropeData)
else:
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), fileEuropeData)
print("Europe fibre file data shape:", dfEurope.shape)

if os.path.exists(fileEuropeHouseholds) :
    print("Reading Europe fibre data (households).....")
    dfEuropeHH = pd.read_csv(fileEuropeHouseholds)
else:
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), fileEuropeHouseholds)
print("Europe fibre file data (households) shape:", dfEuropeHH.shape)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reading Europe fibre data.....
Europe fibre file data shape: (1650, 16)
Reading Europe fibre data (households).....
Europe fibre file data (households) shape: (1650, 15)


The data fields holding the values for each year require some cleaning.

This is done for both dataframes -
dfEurope and dfEuropeHH

1. The % and absolute values are loaded as strings because they contain commas and % signs. There are also some dashes (-) and spaces.
2. The column called Geography level is renamed to URClass to reflect Urban/Rural values

The resulting cleaned dataframes are named

dfEuropeClean

dfEuropeCleanHH

In [22]:
###DEFINE FUNCTIONS FOR CLEANING THE FILE
#fn_clean_years removed % sign and changes datatype to float
#fn_change_col_name will change any column name in any dataframe


def fn_clean_years(thisdf, thiscol):
  #check column is still a string
  if thisdf[thiscol].dtype == 'object':
    #check for dashes -
    thisdf[thiscol] = thisdf[thiscol].str.replace('-', '')
    #convert the yearly value to a float - first remove the % sign
    thisdf[thiscol] = thisdf[thiscol].str.replace('%', '')#.astype(float)
    #check if there's a comma and remove those
    thisdf[thiscol] = thisdf[thiscol].str.replace(',', '')#.astype(float)
    #check for blanks
    thisdf[thiscol] = thisdf[thiscol].str.replace(' ', '')#.astype(float)
    #finally replace empty strings with None
    thisdf[thiscol] = thisdf[thiscol].replace('', None)
    #now change the datatype of the column
    #thisdf.astype({thiscol: 'float'}).dtypes
    thisdf[thiscol] = thisdf[thiscol].astype(float)
  return thisdf

def fn_change_col_name(thisdf, oldname, newname):
  #change the name of the geography column to URClass
  thisdf.rename(columns={oldname: newname}, inplace=True)
  return thisdf

In [23]:
#clean and prepare data
#we need to rename the Geography level URClass cos it didn't seemto like the space in the name
#then for each of the year columns strip the % sign from the value and change it from a string to a float
dfEuropeClean = fn_change_col_name(dfEurope, 'Geography level', 'URClass')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2023')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2022')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2021')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2020')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2019')
dfEuropeClean = fn_clean_years(dfEuropeClean, '2018')
print(dfEuropeClean.head())

#now do the same for the households data
#in this case, we need to strip a , from the year columns and change it to a float
dfEuropeCleanHH = fn_change_col_name(dfEuropeHH, 'Geography level', 'URClass')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2023')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2022')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2021')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2020')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2019')
dfEuropeCleanHH = fn_clean_years(dfEuropeCleanHH, '2018')
print(dfEuropeCleanHH.head())

   Country                        Metric URClass             Unit  \
0  Austria                     Land area   Total       km squared   
1  Austria                    Population   Total              ###   
2  Austria                    Households   Total              ###   
3  Austria   Broadband coverage (>2Mbps)   Total  % of Households   
4  Austria  Broadband coverage (>30Mbps)   Total  % of Households   

           2013          2014          2015          2016          2017  \
0    8387900.0%    8387900.0%    8387900.0%    8387900.0%    8387900.0%   
1  844301800.0%  845186000.0%  850688900.0%  857626100.0%  869007600.0%   
2  367087600.0%  373890627.2%  381326121.0%  385514998.2%  390333256.0%   
3         98.6%         98.4%         98.0%         98.1%         97.9%   
4         55.6%         60.3%         65.2%         67.3%         71.3%   

          2018         2019         2020         2021         2022  \
0    8387900.0    8387900.0    8387900.0    8392700.0    8392700

Reduce the columns to just show those for 2018 onwards, as well as the country, the metric(FTTP etc) and the URClass (Rural or Total)

In [24]:
europeCols = ['Country','Metric','URClass','2018','2019','2020','2021','2022','2023']
dfEuropeClean = dfEuropeClean[europeCols]
print(dfEuropeClean.head())

europeHHCols = ['Country','Metric','URClass','2018','2019','2020','2021','2022','2023']
dfEuropeCleanHH = dfEuropeCleanHH[europeCols]
print(dfEuropeCleanHH.head())

   Country                        Metric URClass         2018         2019  \
0  Austria                     Land area   Total    8387900.0    8387900.0   
1  Austria                    Population   Total  877286500.0  885877500.0   
2  Austria                    Households   Total  393553380.2  388331200.0   
3  Austria   Broadband coverage (>2Mbps)   Total         98.1         98.2   
4  Austria  Broadband coverage (>30Mbps)   Total         72.4         78.8   

          2020         2021         2022         2023  
0    8387900.0    8392700.0    8392700.0    8392700.0  
1  890106400.0  893266400.0  897892900.0  910477200.0  
2  391892900.0  395914300.0  399505000.0  403308000.0  
3         98.6          NaN          NaN          NaN  
4         86.6         93.3         94.8         94.2  
   Country                        Metric URClass       2018       2019  \
0  Austria                     Land area   Total    83879.0    83879.0   
1  Austria                    Population   Tota

Now rename the value fields in each dataset so when merged they have sensible names

in the percentage dataframe
2018 becomes 2018% etc

in the no of households dataframe
2018 becomes 2018HH etc

Then merge the two dataframes on Country/Metric/URClass

In [25]:
#rename year columns in each dataset
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2018', '2018%')
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2019', '2019%')
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2020', '2020%')
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2021', '2021%')
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2022', '2022%')
dfEuropeClean = fn_change_col_name(dfEuropeClean, '2023', '2023%')

dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2018', '2018HH')
dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2019', '2019HH')
dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2020', '2020HH')
dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2021', '2021HH')
dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2022', '2022HH')
dfEuropeCleanHH = fn_change_col_name(dfEuropeCleanHH, '2023', '2023HH')

##merge the two datasets
colslist = ['Country', 'Metric','URClass']
dfFinal = pd.merge(dfEuropeClean,dfEuropeCleanHH,on=colslist, how='inner')



<ipython-input-22-5281cde9183b>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  thisdf.rename(columns={oldname: newname}, inplace=True)


Finally, lets only return those rows for the metric FTTP - this means we will have only values for fibre to the premises availability

The resulting dataframe dfEuropeCleanFTTP, has the following structure

Country - name of the country

Metric - now always FTTP

URClass - Total (ie rural and urban) or Rural

2018% - percentage of households with access to FTTP in 2018

2018HH - absolute number of households with access to FTTP in 2018

2019% - percentage of households with access to FTTP in 2019

2019HH - absolute number of households with access to FTTP in 2019

2020% - percentage of households with access to FTTP in 2020

2020HH - absolute number of households with access to FTTP in 2020

2021% - percentage of households with access to FTTP in 2021

2021HH - absolute number of households with access to FTTP in 2021

2022% - percentage of households with access to FTTP in 2022

2022HH - absolute number of households with access to FTTP in 2022

2023% - percentage of households with access to FTTP in 2023

2023HH - absolute number of households with access to FTTP in 2023



In [26]:
###DEFINE FUNCTION TO FILTER DATASET BY METRIC

def fn_filter_by_metric(thisdf, thismetric):
  #return a new dataset which only contains specified metric
  return thisdf.query('Metric == "' + thismetric + '"')

#now run this function to get only those rows where Metric = FTTP
#this will give us all the rows for full fibre availability
dfEuropeCleanFTTP = fn_filter_by_metric(dfFinal, 'FTTP')
dfEuropeCleanFTTP.head()

#drop summary rows so only the countries are left
dfEuropeCleanFTTPTotalDropEU = dfEuropeCleanFTTPTotal.drop(dfEuropeCleanFTTPTotal[dfEuropeCleanFTTPTotal['Country'] == 'EU27'].index)
dfEuropeCleanFTTPTotalDropEU = dfEuropeCleanFTTPTotalDropEU.drop(dfEuropeCleanFTTPTotal[dfEuropeCleanFTTPTotal['Country'] == 'EU28'].index)


Now we need to get some summative and descriptive numbers
for the countries
- top 5 countries in 2023 for availability % (urban and rural)
- bottom 5 countries in 2023 for availability % (urban and rural)


- top 5 countries in 2023 for availability % (rural only)
- bottom 5 countries in 2023 for availability % (rural only)


-  5 countries with highest growth in availability % in last 5 years (urban and rural)
-  5 countries with lowest  growth in availability % in last 5 years (urban and rural)


In [27]:
##############################################################
#INITIAL ANALYSIS###############################
##############################################################

####Funtion to filter dataset by URClass

def fn_filter_by_urclass(thisdf, thisUrclass):
  #return a new dataset which only contains specified URClass
  return thisdf.query('URClass == "' + thisUrclass + '"')

####Funtion to filter dataset by country
def fn_filter_by_country(thisdf, thiscountry):
  #return a new dataset which only contains specified country
  return thisdf.query('Country == "' + thiscountry + '"')



In [28]:

#order by 2023% column to set which 5 countries have the highest % availability in 2023
#show top five only
#filter to get total rows only
dfEuropeCleanFTTPTotal = fn_filter_by_urclass(dfEuropeCleanFTTP, 'Total')

dfEuropeCleanFTTPTotal = dfEuropeCleanFTTPTotal.sort_values(by=['2023%'], ascending=False)
print(dfEuropeCleanFTTPTotal.iloc[0:5])

#now show those with the largest amount of population with access to FTTP
dfEuropeCleanFTTPTotal = dfEuropeCleanFTTPTotal.sort_values(by=['2023HH'], ascending=False)
print(dfEuropeCleanFTTPTotal.iloc[0:5])

      Country Metric URClass  2018%  2019%  2020%  2021%  2022%  2023%  \
798     Spain   FTTP   Total   77.4   80.4   84.9   88.9   91.0   95.2   
711   Romania   FTTP   Total   62.7   68.1   75.9   87.1   95.6   95.0   
682  Portugal   FTTP   Total   70.2   76.6   82.3   87.6   90.8   92.3   
392   Iceland   FTTP   Total   76.2   80.4   83.5   87.6   88.2   91.0   
73   Bulgaria   FTTP   Total   54.2   65.2   75.2   81.4   85.6   88.6   

         2018HH      2019HH      2020HH      2021HH      2022HH      2023HH  
798  13907642.0  14487413.0  15442337.0  16304471.0  16710576.0  17492229.0  
711   4694397.0   5098126.0   5681433.0   6515089.0   7150980.0   7106377.0  
682   2850981.0   3100505.0   3319290.0   3540247.0   3668679.0   3749559.0  
392    100829.0    114011.0    118376.0    122084.0    120464.0    126890.0  
73    1589576.0   1877295.0   2173209.0   2345497.0   2439632.0   2484082.0  
            Country Metric URClass  2018%  2019%  2020%  2021%  2022%  2023%  \
943    

In [29]:
#order by 2023 column to set which 5 countries have the lowest % availability
#show bottom five only
#filter to get total rows only
dfEuropeCleanFTTPTotal = fn_filter_by_urclass(dfEuropeCleanFTTP, 'Total')

dfEuropeCleanFTTPTotal = dfEuropeCleanFTTPTotal.sort_values(by=['2023%'], ascending=True)
print(dfEuropeCleanFTTPTotal.iloc[0:5])


     Country Metric URClass  2018%  2019%  2020%  2021%  2022%  2023%  \
44   Belgium   FTTP   Total    1.4    3.6    6.5   10.1   17.2   25.0   
305  Germany   FTTP   Total    8.5   10.5   13.8   15.4   19.3   29.8   
160  Czechia   FTTP   Total   28.3   29.3   33.3   35.8   37.4   36.0   
334   Greece   FTTP   Total    0.4    7.1   10.2   19.8   27.8   38.4   
15   Austria   FTTP   Total   13.0   13.8   20.5   26.6   36.6   41.0   

        2018HH     2019HH     2020HH     2021HH     2022HH      2023HH  
44     68689.0   174923.0   309472.0   503257.0   861948.0   1204619.0  
305  3463230.0  4351459.0  5731176.0  6425303.0  7975905.0  12325987.0  
160  1247049.0  1296949.0  1405288.0  1515831.0  1587877.0   1734901.0  
334    16624.0   303044.0   436289.0   849460.0  1189474.0   1606944.0  
15    512932.0   534791.0   805015.0  1054017.0  1463133.0   1652409.0  


In [30]:
#order by 2023 column to set which 5 countries have the highest % availability in 2023 but for Rural areas only
#show top five only
#filter to get total rows only
dfEuropeCleanFTTPRural = fn_filter_by_urclass(dfEuropeCleanFTTP, 'Rural')

dfEuropeCleanFTTPRural = dfEuropeCleanFTTPRural.sort_values(by=['2023%'], ascending=False)
print(dfEuropeCleanFTTPRural.iloc[0:5])


          Country Metric URClass  2018%  2019%  2020%  2021%  2022%  2023%  \
1469      Romania   FTTP   Rural   30.1   39.1   55.6   75.7   90.1   92.5   
1091      Denmark   FTTP   Rural   60.8   65.8   70.9   77.8   87.0   90.3   
1532        Spain   FTTP   Rural   32.6   46.4   59.5   68.9   73.9   85.9   
1238      Iceland   FTTP   Rural   39.4   54.7   66.3   78.4   78.7   83.9   
1385  Netherlands   FTTP   Rural   22.7   26.4   27.2   54.5   76.4   78.4   

         2018HH     2019HH     2020HH     2021HH     2022HH     2023HH  
1469   473059.0   613199.0   873030.0  1188700.0  1413820.0  1451283.0  
1091   178314.0   193084.0   207965.0   227507.0   254218.0   262890.0  
1532  1073626.0  1512794.0  1850542.0  2109898.0  2260398.0  2623263.0  
1238     3127.0     5459.0     6616.0     5493.0     4660.0     5081.0  
1385    68127.0    82558.0    86650.0   173302.0   243072.0   250693.0  


In [31]:
#order by 2023 column to set which 5 countries have the lowest % availability in 2023 but for Rural areas only
#show bottom five only
#filter to get total rows only
dfEuropeCleanFTTPTotal = fn_filter_by_urclass(dfEuropeCleanFTTP, 'Rural')

dfEuropeCleanFTTPTotal = dfEuropeCleanFTTPTotal.sort_values(by=['2023%'], ascending=True)
print(dfEuropeCleanFTTPTotal.iloc[0:5])


      Country Metric URClass  2018%  2019%  2020%  2021%  2022%  2023%  \
1196   Greece   FTTP   Rural    0.0    0.0    0.0    0.0    0.0    0.0   
1070  Czechia   FTTP   Rural    5.6    5.9    6.4    6.9    8.1    7.2   
986   Belgium   FTTP   Rural    0.0    0.1    0.4    0.7    1.3    7.3   
1322   Latvia   FTTP   Rural    8.9    9.2   10.0   11.0   11.2   11.7   
1364    Malta   FTTP   Rural    0.0    0.0    0.0    0.0    0.0   16.2   

       2018HH   2019HH   2020HH   2021HH   2022HH   2023HH  
1196      NaN      NaN     50.0      NaN      NaN      NaN  
1070  35045.0  36621.0  40186.0  43098.0  48097.0  67394.0  
986      59.0    299.0    788.0   1861.0   3281.0  15133.0  
1322  18606.0  19843.0  20518.0  20894.0  21274.0  22011.0  
1364      NaN      NaN      NaN      NaN      NaN    179.0  


In [32]:
#EUROPE-WIDE MEAN % AVAILABILITY IN 2023

averageFTTP = round(dfEuropeCleanFTTPTotalDropEU['2023%'].mean(),2)
print('Mean FTTP availability (%) in 2023 : ',
averageFTTP, '%')

Mean FTTP availability (%) in 2023 :  69.1 %


In [36]:
#highest growth between 2019 and 2023
#filter to get total rows only
dfEuropeCleanFTTPTotal = fn_filter_by_urclass(dfEuropeCleanFTTP, 'Total')
#create a new column for each row which is the percentage growth betwen 2019 and 2023
#this is calculated as:
#(2023 avail % - 2019 avail %) to give us the difference
# then calculate that difference as a percentage of the original figure

#NB some rows have 0 % availability. If this is the case, set it to 0.1 to allow caclulation to be done
#if value is 0 then set it to 0.1
dfEuropeCleanFTTPTotal['2019%'] = dfEuropeCleanFTTPTotal['2019%'].replace(0, 0.1)
results_df = dfEuropeCleanFTTPTotal
results_df['5 year Growth (%)'] = round(((results_df['2023%']-results_df['2019%'])/results_df['2019%'])*100,2)

#results_final_df = pd.DataFrame(results_df['Country'],['5 year Growth'],['2019'],['2023'])
#results_final_df = results_final_df.sort_values(by=['5 year Growth'], ascending=False)
final_results_df = results_df.filter(['Country','5 year Growth (%)','2019%','2023%'])
final_results_df = final_results_df.sort_values(by=['5 year Growth (%)'], ascending=False)
print("Highest growth")
print(final_results_df.iloc[0:5])
final_results_df = final_results_df.sort_values(by=['5 year Growth (%)'], ascending=True)
print("Lowest growth")
print(final_results_df.iloc[0:5])


Highest growth
            Country  5 year Growth (%)  2019%  2023%
131          Cyprus             663.37   10.1   77.1
44          Belgium             594.44    3.6   25.0
885  United Kingdom             507.06    8.5   51.6
334          Greece             440.85    7.1   38.4
15          Austria             197.10   13.8   41.0
Lowest growth
        Country  5 year Growth (%)  2019%  2023%
508      Latvia               6.17   58.3   61.9
827      Sweden               8.82   77.1   83.9
392     Iceland              13.18   80.4   91.0
537  Luxembourg              16.89   67.5   78.9
798       Spain              18.41   80.4   95.2


<ipython-input-36-f4f51c3ae59e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfEuropeCleanFTTPTotal['2019%'] = dfEuropeCleanFTTPTotal['2019%'].replace(0, 0.1)
<ipython-input-36-f4f51c3ae59e>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['5 year Growth (%)'] = round(((results_df['2023%']-results_df['2019%'])/results_df['2019%'])*100,2)
